In [17]:
import sqlite3
con = sqlite3.connect("bot18.db")
cur = con.cursor()

In [18]:
cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT, 
        seen BOOLEAN DEFAULT FALSE,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
""")

con.commit()

In [19]:
import requests
import time

def download(method,url,
            param=None, data=None , timeout= 1, maxretries = 3):

    try:
        resp = requests.request(method, url,
                               params= param, data= data,
                                headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
#         print(e.code)
#         print(e.reason)
        if 500<= e.response.status_code <600 and maxretries >0:
            print(maxretries)
            time.sleep(timeout)
            download(method,url,param,data,timeout,maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [20]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    return [requests.compat.urljoin(seed, _["href"])
 for _ in dom.find_all("a") 
        if _.has_attr("href") and len(_["href"]) > 3]

In [21]:
# seed URL -> DB Insert
#큐생성
url = "https://www.google.com/search"
param = {"q":"박보영"}
html = download("get",url,param)
dom = BeautifulSoup(html.text, "lxml")

#url 하나씩 꺼내서 parse 서로다른 table에 넣을거기때문에
for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
    _urlparse = requests.compat.urlparse(href)
    
    #schema랑 net loacation이랑
    netloc = "://".join(_urlparse[:2])
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
    
    netlocID = cur.fetchone()
    if not netlocID:
        cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
        con.commit()
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
        
        cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)",
                   [netlocID[0], _urlparse[2], _urlparse[4]])
        con.commit()

In [22]:

while True:
    cur.execute("""
        SELECT table1.id, table2.netloc, table1.path, table1.param 
        FROM table1 JOIN table2 
        ON table1.table2_id = table2.id 
        WHERE table1.seen = FALSE
        ORDER BY table1.date ASC
        LIMIT 0,1;
    """)
    seed = cur.fetchone()
    if not seed:
        break;
    
    cur.execute("""
        UPDATE table1 SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
    
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    print(baseURL)
    for href in parseURL(baseURL):
        _urlparse = requests.compat.urlparse(href)
        netloc = "://".join(_urlparse[:2])
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])

        netlocID = cur.fetchone()
        if not netlocID:
            cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
            con.commit()
            cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
            netlocID = cur.fetchone()

        cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)",
                   [netlocID[0], _urlparse[2], _urlparse[4]])
        con.commit()
        print(cur.lastrowid,netlocID)
    break

https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81?
9 (1,)
10 (1,)
11 (1,)
12 (9,)
13 (1,)
14 (1,)
15 (1,)
16 (1,)
17 (1,)
18 (1,)
19 (1,)
20 (1,)
21 (1,)
22 (1,)
23 (10,)
24 (10,)
25 (10,)
26 (10,)
27 (10,)
28 (10,)
29 (10,)
30 (10,)
31 (10,)
32 (10,)
33 (10,)
34 (10,)
35 (1,)
36 (1,)
37 (1,)
38 (1,)
39 (1,)
40 (1,)
41 (10,)
42 (1,)
43 (1,)
44 (1,)
45 (1,)
46 (1,)
47 (1,)
48 (1,)
49 (1,)
50 (1,)
51 (1,)
52 (1,)
53 (1,)
54 (1,)
55 (1,)
56 (1,)
57 (1,)
58 (1,)
59 (1,)
60 (1,)
61 (1,)
62 (1,)
63 (1,)
64 (1,)
65 (1,)
66 (1,)
67 (1,)
68 (1,)
69 (1,)
70 (1,)
71 (1,)
72 (1,)
73 (1,)
74 (1,)
75 (1,)
76 (1,)
77 (1,)
78 (1,)
79 (11,)
80 (1,)
81 (12,)
82 (1,)
83 (13,)
84 (1,)
85 (1,)
86 (1,)
87 (1,)
88 (1,)
89 (1,)
90 (1,)
91 (1,)
92 (1,)
93 (1,)
94 (1,)
95 (1,)
96 (1,)
97 (1,)
98 (1,)
99 (1,)
100 (1,)
101 (1,)
102 (1,)
103 (1,)
104 (1,)
105 (1,)
106 (1,)
107 (1,)
108 (1,)
109 (1,)
110 (1,)
111 (1,)
112 (1,)
113 (1,)
114 (1,)
115 (1,)
116 (1,)
117 (1,)
118 (1,)
119 (1,)
120 (1,)
121 (1,)
122 (1,)
1

1009 (12,)
1010 (1,)
1011 (1,)
1012 (12,)
1013 (1,)
1014 (12,)
1015 (1,)
1016 (12,)
1017 (1,)
1018 (12,)
1019 (1,)
1020 (12,)
1021 (1,)
1022 (12,)
1023 (1,)
1024 (12,)
1025 (1,)
1026 (12,)
1027 (1,)
1028 (12,)
1029 (1,)
1030 (1,)
1031 (1,)
1032 (1,)
1033 (1,)
1034 (1,)
1035 (16,)
1036 (1,)
1037 (1,)
1038 (16,)
1039 (16,)
1040 (16,)
1041 (1,)
1042 (1,)
1043 (18,)
1044 (1,)
1045 (1,)
1046 (18,)
1047 (18,)
1048 (18,)
1049 (16,)
1050 (1,)
1051 (1,)
1052 (18,)
1053 (18,)
1054 (1,)
1055 (1,)
1056 (16,)
1057 (1,)
1058 (16,)
1059 (18,)
1060 (1,)
1061 (18,)
1062 (18,)
1063 (18,)
1064 (1,)
1065 (18,)
1066 (18,)
1067 (18,)
1068 (18,)
1069 (18,)
1070 (18,)
1071 (1,)
1072 (16,)
1073 (18,)
1074 (18,)
1075 (18,)
1076 (18,)
1077 (1,)
1078 (1,)
1079 (16,)
1080 (1,)
1081 (1,)
1082 (16,)
1083 (1,)
1084 (1,)
1085 (1,)
1086 (1,)
1087 (1,)
1088 (18,)
1089 (18,)
1090 (18,)
1091 (1,)
1092 (1,)
1093 (1,)
1094 (18,)
1095 (18,)
1096 (18,)
1097 (1,)
1098 (1,)
1099 (1,)
1100 (16,)
1101 (1,)
1102 (1,)
1103 (1,)
110

In [23]:
select inbound, count(inbound)
from table1 where table2_id=2 group by inbound;

SyntaxError: invalid syntax (<ipython-input-23-d46495e3ab79>, line 1)

In [26]:
cur.execute("""
            select 'inbound', count('inbound')
            from table1
            where table2_id=2 group by 'inbound'
            """)
cur.fetchall()

[('inbound', 1)]

In [27]:
cur.execute("""
    SELECT inbound, count(inbound)
    FROM table1
    where table2_id = 7 group by inbound;
""")
cur.fetchall()

OperationalError: no such column: inbound